In [5]:
import numpy as np
import pandas as pd
import re
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import Model 
from keras.layers import Input, Activation, Embedding, LSTM
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical


In [8]:
df = pd.read_csv('donda.csv', encoding='latin1')


In [9]:
text = df

In [10]:
 tokenizer = Tokenizer()
 tokenizer.fit_on_texts(text)
 vocab_size = len(tokenizer.word_counts) + 1
 print("Total Vocabulary: ", vocab_size )

Total Vocabulary:  6


In [11]:
 N = text.shape[0]
 print("Total Row Count: ", N)
 prop_train = 0.8
 train = int(N*prop_train)
 print("Training Data Count: ", train)
 test = N - train
 print("Test Data Coount: ", test)

Total Row Count:  6168
Training Data Count:  4934
Test Data Coount:  1234


In [12]:
sequences, index_train, index_test = [], [], []
count = 0
for irow,line in enumerate(text):
    #print(irow, line)
    encoded = tokenizer.texts_to_sequences([line])[0]    
    #print(encoded)
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        if irow < train:        
            index_train.append(count)     
        else:         
            index_test.append(count)
        count += 1
print('Total Sequences: %d' % (len(sequences)))

Total Sequences: 4


In [13]:
from keras_preprocessing.sequence import pad_sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % (max_length))

Max Sequence Length: 5


In [14]:
sequences = np.array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
print(y.shape)
y = to_categorical(y, num_classes=vocab_size)
X_train, y_train, X_test, y_test = X[index_train], y[index_train],X[index_test],  y[index_test]
print(X_train.shape)
print(y_train.shape)
#make it smaller for testing
X_train = X_train[:1000]
y_train = y_train[:1000]
X_test = X_test[:100]
y_test = y_test[:100]

(4,)
(4, 4)
(4, 6)


In [15]:
def build_model(vocab_size,
                  input_length=1,
                  dim_dense_embedding=10,
                  hidden_unit_LSTM=5):
    main_input = Input(shape=(input_length,),dtype='int32',name='main_input')
    embedding = Embedding(vocab_size, dim_dense_embedding, 
                          input_length=input_length)(main_input)
    x = LSTM(hidden_unit_LSTM)(embedding)
    main_output = Dense(vocab_size, activation='softmax')(x)
    model = Model(inputs=[main_input],
                   outputs=[main_output])
    return(model)

In [16]:
model = build_model(vocab_size,
                   input_length=X.shape[1],
                   dim_dense_embedding=30,
                   hidden_unit_LSTM=64)
 ##compile network
model.compile(loss='categorical_crossentropy', 
               optimizer='adam', metrics=['accuracy'])
 ##fit network
tf_model = model.fit(X_train, y_train, 
                  validation_data = (X_test,y_test),
                  epochs=20, verbose=2,
                  batch_size=128)

Epoch 1/20
1/1 - 2s - loss: 1.7908 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/20
1/1 - 0s - loss: 1.7858 - accuracy: 0.0000e+00 - 33ms/epoch - 33ms/step
Epoch 3/20
1/1 - 0s - loss: 1.7806 - accuracy: 0.0000e+00 - 30ms/epoch - 30ms/step
Epoch 4/20
1/1 - 0s - loss: 1.7755 - accuracy: 0.2500 - 36ms/epoch - 36ms/step
Epoch 5/20
1/1 - 0s - loss: 1.7702 - accuracy: 0.2500 - 32ms/epoch - 32ms/step
Epoch 6/20
1/1 - 0s - loss: 1.7647 - accuracy: 0.2500 - 33ms/epoch - 33ms/step
Epoch 7/20
1/1 - 0s - loss: 1.7591 - accuracy: 0.2500 - 27ms/epoch - 27ms/step
Epoch 8/20
1/1 - 0s - loss: 1.7533 - accuracy: 0.5000 - 30ms/epoch - 30ms/step
Epoch 9/20
1/1 - 0s - loss: 1.7473 - accuracy: 0.5000 - 27ms/epoch - 27ms/step
Epoch 10/20
1/1 - 0s - loss: 1.7411 - accuracy: 0.5000 - 28ms/epoch - 28ms/step
Epoch 11/20
1/1 - 0s - loss: 1.7345 - accuracy: 0.5000 - 31ms/epoch - 31ms/step
Epoch 12/20
1/1 - 0s - loss: 1.7277 - accuracy: 0.5000 - 29ms/epoch - 29ms/step
Epoch 13/20
1/1 - 0s - loss: 1.7204 - acc

In [17]:
model.load_weights('donda.h5')

ValueError: Invalid bias shape: (2, 3072)

In [ ]:
t